# Importing, Discovering, Cleaning, API Calls, Merging, Reformatting

In [1]:
# Set Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings
import os
import json
import requests
import time
from api_key import dark_sky_key
warnings.filterwarnings('ignore')

### Importing the NYC Traffic Data CSV

In [2]:
# Importing the CSV File from NYC
raw_data = os.path.join('..', 'data', 'rawdata', 'NYPD_Motor_Vehicle_Collisions.csv')
nyc_data = pd.read_csv(raw_data)
# Preview the DataFrame
nyc_data.head()

,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,UNIQUE KEY,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,03/16/2019,0:00,BROOKLYN,11208,40.681927,-73.870720,"(40.681927, -73.87072)",ATLANTIC AVENUE,HEMLOCK STREET,NaN,...,NaN,NaN,NaN,NaN,4098779,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN
1,03/16/2019,0:00,MANHATTAN,10016,40.749100,-73.984085,"(40.7491, -73.984085)",5 AVENUE,WEST 35 STREET,NaN,...,Driver Inattention/Distraction,NaN,NaN,NaN,4097559,Sedan,Garbage or Refuse,NaN,NaN,NaN
2,03/16/2019,0:00,QUEENS,11412,40.696064,-73.763084,"(40.696064, -73.763084)",MEXICO STREET,DORMANS ROAD,NaN,...,Unspecified,NaN,NaN,NaN,4098023,Sedan,Sedan,NaN,NaN,NaN
3,03/16/2019,0:00,NaN,NaN,40.841843,-73.945390,"(40.841843, -73.94539)",HENRY HUDSON PARKWAY,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4098377,Sedan,Taxi,NaN,NaN,NaN
4,03/16/2019,0:00,BRONX,10451,40.811733,-73.926700,"(40.811733, -73.9267)",MORRIS AVENUE,EAST 139 STREET,NaN,...,Unspecified,NaN,NaN,NaN,4097667,Sedan,Sedan,NaN,NaN,NaN


### Examining The CSV Data

In [3]:
#Examine data columns
#nyc_data.columns

In [4]:
#check data types
#nyc_data.dtypes

In [5]:
# Explore amount of unique records 
#nyc_data.nunique()

### Begin Data Consolidation For Project

In [6]:
# Concat date and time columns for easy analysis
nyc_data['COLLISION DATE'] = nyc_data['DATE'] + ' ' + nyc_data['TIME']
nyc_data['COLLISION DATE'] = pd.to_datetime(nyc_data['COLLISION DATE'])
# Add UNIX Date to pass into Dark Sky API
nyc_data['UNIX DATE'] = nyc_data['COLLISION DATE'].astype(np.int64) // 10**9
nyc_data.head()

,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,UNIQUE KEY,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,COLLISION DATE,UNIX DATE
0,03/16/2019,0:00,BROOKLYN,11208,40.681927,-73.870720,"(40.681927, -73.87072)",ATLANTIC AVENUE,HEMLOCK STREET,NaN,...,NaN,NaN,4098779,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN,2019-03-16,1552694400
1,03/16/2019,0:00,MANHATTAN,10016,40.749100,-73.984085,"(40.7491, -73.984085)",5 AVENUE,WEST 35 STREET,NaN,...,NaN,NaN,4097559,Sedan,Garbage or Refuse,NaN,NaN,NaN,2019-03-16,1552694400
2,03/16/2019,0:00,QUEENS,11412,40.696064,-73.763084,"(40.696064, -73.763084)",MEXICO STREET,DORMANS ROAD,NaN,...,NaN,NaN,4098023,Sedan,Sedan,NaN,NaN,NaN,2019-03-16,1552694400
3,03/16/2019,0:00,NaN,NaN,40.841843,-73.945390,"(40.841843, -73.94539)",HENRY HUDSON PARKWAY,NaN,NaN,...,NaN,NaN,4098377,Sedan,Taxi,NaN,NaN,NaN,2019-03-16,1552694400
4,03/16/2019,0:00,BRONX,10451,40.811733,-73.926700,"(40.811733, -73.9267)",MORRIS AVENUE,EAST 139 STREET,NaN,...,NaN,NaN,4097667,Sedan,Sedan,NaN,NaN,NaN,2019-03-16,1552694400


In [7]:
# Making sure the dates are correct
nyc_data['COLLISION DATE'].min()

Timestamp('2012-07-01 00:05:00')

In [8]:
nyc_data['COLLISION DATE'].max()

Timestamp('2019-03-16 23:55:00')

In [9]:
# Dropping Dates before January 1st 2014, and after January 1st, 2019 so that data only consists of 01-01-2014-12-31-2018
# This makes the data more uniform so patterns can become more obvious
drp_nyc_data = nyc_data[(nyc_data['COLLISION DATE'] > '2014-01-01') & (nyc_data['COLLISION DATE'] < '2019-01-01')]
drp_nyc_data.head()

,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,UNIQUE KEY,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,COLLISION DATE,UNIX DATE
341,12/30/2018,12:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4054741,Sedan,Sedan,NaN,NaN,NaN,2018-12-30 12:00:00,1546171200
830,12/30/2018,12:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4054750,Sedan,Sedan,NaN,NaN,NaN,2018-12-30 12:00:00,1546171200
7426,12/28/2018,0:00,NaN,NaN,NaN,NaN,NaN,BROOKLYN QUEENS EXPRESSWAY,NaN,NaN,...,NaN,NaN,4054018,Tractor Truck Diesel,Sedan,NaN,NaN,NaN,2018-12-28 00:00:00,1545955200
7650,12/17/2018,16:45,NaN,NaN,NaN,NaN,NaN,VERRAZANO BRIDGE,NaN,NaN,...,NaN,NaN,4047586,Sedan,Sedan,NaN,NaN,NaN,2018-12-17 16:45:00,1545065100
8900,12/11/2018,13:00,NaN,NaN,NaN,NaN,NaN,GOWANUS RAMP,NaN,NaN,...,NaN,NaN,4042911,Sedan,Sedan,NaN,NaN,NaN,2018-12-11 13:00:00,1544533200


In [10]:
# Verifying 2013 and 2019 dates do not appear
#coll_by_yr = drp_nyc_data['DATE'].value_counts().sort_index()
#coll_by_yr

#### Creating Master NYC Traffic Collision DataFrame

In [11]:
#Rename Two Columns
renamed = drp_nyc_data.rename(columns={"NUMBER OF PERSONS INJURED":"INJURIES", "NUMBER OF PERSONS KILLED":"DEATHS"})

In [12]:
renamed['STAN DATE'] = pd.to_datetime(renamed['UNIX DATE'],unit='s')
renamed.head()

,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 5,UNIQUE KEY,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,COLLISION DATE,UNIX DATE,STAN DATE
341,12/30/2018,12:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4054741,Sedan,Sedan,NaN,NaN,NaN,2018-12-30 12:00:00,1546171200,2018-12-30 12:00:00
830,12/30/2018,12:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4054750,Sedan,Sedan,NaN,NaN,NaN,2018-12-30 12:00:00,1546171200,2018-12-30 12:00:00
7426,12/28/2018,0:00,NaN,NaN,NaN,NaN,NaN,BROOKLYN QUEENS EXPRESSWAY,NaN,NaN,...,NaN,4054018,Tractor Truck Diesel,Sedan,NaN,NaN,NaN,2018-12-28 00:00:00,1545955200,2018-12-28 00:00:00
7650,12/17/2018,16:45,NaN,NaN,NaN,NaN,NaN,VERRAZANO BRIDGE,NaN,NaN,...,NaN,4047586,Sedan,Sedan,NaN,NaN,NaN,2018-12-17 16:45:00,1545065100,2018-12-17 16:45:00
8900,12/11/2018,13:00,NaN,NaN,NaN,NaN,NaN,GOWANUS RAMP,NaN,NaN,...,NaN,4042911,Sedan,Sedan,NaN,NaN,NaN,2018-12-11 13:00:00,1544533200,2018-12-11 13:00:00


In [13]:
renamed['YEAR'] = renamed['COLLISION DATE'].dt.year
renamed['MONTH'] = renamed['COLLISION DATE'].dt.month
renamed.head()

,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,COLLISION DATE,UNIX DATE,STAN DATE,YEAR,MONTH
341,12/30/2018,12:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Sedan,Sedan,NaN,NaN,NaN,2018-12-30 12:00:00,1546171200,2018-12-30 12:00:00,2018,12
830,12/30/2018,12:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Sedan,Sedan,NaN,NaN,NaN,2018-12-30 12:00:00,1546171200,2018-12-30 12:00:00,2018,12
7426,12/28/2018,0:00,NaN,NaN,NaN,NaN,NaN,BROOKLYN QUEENS EXPRESSWAY,NaN,NaN,...,Tractor Truck Diesel,Sedan,NaN,NaN,NaN,2018-12-28 00:00:00,1545955200,2018-12-28 00:00:00,2018,12
7650,12/17/2018,16:45,NaN,NaN,NaN,NaN,NaN,VERRAZANO BRIDGE,NaN,NaN,...,Sedan,Sedan,NaN,NaN,NaN,2018-12-17 16:45:00,1545065100,2018-12-17 16:45:00,2018,12
8900,12/11/2018,13:00,NaN,NaN,NaN,NaN,NaN,GOWANUS RAMP,NaN,NaN,...,Sedan,Sedan,NaN,NaN,NaN,2018-12-11 13:00:00,1544533200,2018-12-11 13:00:00,2018,12


In [14]:
renamed['WEEK'] = renamed['STAN DATE'].dt.week
renamed['YEAR WEEK'] = renamed['YEAR'].map(str) + "/" + renamed['WEEK'].map(str)
renamed['YEAR MONTH'] = renamed['YEAR'].map(str) + "/" + renamed['MONTH'].map(str)

In [15]:
# Creating a new DataFrame that removes the unwanted columns
final_nyc_data = renamed[['DATE', 'YEAR MONTH', 'YEAR WEEK', 'YEAR', 'MONTH','WEEK', 'UNIX DATE', 'DEATHS', 'INJURIES']]
final_nyc_data.head()

,DATE,YEAR MONTH,YEAR WEEK,YEAR,MONTH,WEEK,UNIX DATE,DEATHS,INJURIES
341,12/30/2018,2018/12,2018/52,2018,12,52,1546171200,0.0,0.0
830,12/30/2018,2018/12,2018/52,2018,12,52,1546171200,0.0,0.0
7426,12/28/2018,2018/12,2018/52,2018,12,52,1545955200,0.0,0.0
7650,12/17/2018,2018/12,2018/51,2018,12,51,1545065100,0.0,0.0
8900,12/11/2018,2018/12,2018/50,2018,12,50,1544533200,0.0,0.0


#### Exporting DataFrame to CSV

In [16]:
# Putting this file into master_clean_data folder to avoid conflicts and confusion
final_nyc_data.to_csv(os.path.join('..', 'data', 'master_clean_data', 'master_nyc_collision_data.csv'), index = False, header = True)

# API Calls
For a complete view of the API calls, please reference 2_Chris-DarkSkyAPICall.ipynb, and 2_Ibrahim-DarkSkyAPICall_Part2.ipynb found inside of the notebooks folder. This API can only have 1000 free calls per day per API key. The calls were run iteratively using our unique keys, and then saved as a CSV. We then combined the 3 CSV files, and merged that data into our main dataframe. 


# DO NOT RUN THESE CELLS
#### Process For Gathering Dark Sky Data

In [17]:
# DO NOT RUN THIS CELL
# First Iteration was removed by started on December 31st, 2013
# Second interation seen below
# Starting at 1466186400 or the 16th June as I already api called the data from 2014 to this date previously
#days3 = [1468000800]
#days2 = [1466186400]
#days1 = [1388448000]
#count = 0
#for day in days1:
#    day = day + (24 * 60 * 60)
#    days.append(day)
#    if count >=20:
#        break
#    count +=1
    
#for day in days2:
#    day = day + (24 * 60 * 60)
#    days.append(day)
#    if count >=20:
#        break
#    count +=1
    
#for day in days3:
#    day = day + (24 * 60 * 60)
#    days.append(day)
#    if count >=20:
#        break
#    count +=1

In [18]:
# Checking Days to make sure they're correct for the calls. 
#print("Day Count Iteration 1 " + str(len(days1)))
#days1
#print("Day Count Iteration 2 " + str(len(days1)))
#days2

In [19]:
#time.ctime(int("1466186400"))
#time.strftime("%D %H:%M", time.localtime(int("1466186400")))

In [20]:
# Setting API Data
# Center of NYC Lat Long
# Used this because the large amount of missing Lat Long Data in the DF, and our limited amount of API calls
# If more free API calls were allowed, we would have iterated through all 1 million rows
# Also due to the limited amount of API calls, we only gathered average weather data for the Day, instead of by minute or hour.
#lat = "40.73"
#lng = "-73.99"
#exclude = "currently,flags,alerts,minutely,hourly"
#units = "us"
#url = f'https://api.darksky.net/forecast/{dark_sky_key}/{lat},{lng}'

#### Gathering Data From Dark Sky

In [21]:
# Gathering API Data
# DO NOT RUN
# ADD YOUR API KEY TO API_KEY FILE IN MASTER_NOTEBOOKS API_KEY
# darksky_data_1 = []

# count = 0


#print("Beginning Data Retrieval")
#print("-------------------------------")

#for day in days:
    
#    response = requests.get(f"{url},{day}?exclude={exclude}?units={units}").json()
#    darksky_data_1.append({'Day' : day,
#                           'Summary' : response['daily']['data'][0]['summary'],
#                           'Detail' : response['daily']['data'][0]['icon'],
#                           'Sunrise' : response['daily']['data'][0]['sunriseTime'],
#                           'Sunset' : response['daily']['data'][0]['sunsetTime'],
#                           'Moonphase' : response['daily']['data'][0]['moonPhase'],
#                           'precipIntensity' : response['daily']['data'][0]['precipIntensity'],
#                           'precipIntensityMax' : response['daily']['data'][0]['precipIntensityMax'],
#                           'precipProbability' : response['daily']['data'][0]['precipProbability'],
#                           'temperatureMax' : response['daily']['data'][0]['temperatureMax'],
#                           'temperatureMaxTime' : response['daily']['data'][0]['temperatureMaxTime'],
#                           'temperatureMin' : response['daily']['data'][0]['temperatureMin'],
#                           'temperatureMinTime' : response['daily']['data'][0]['temperatureMinTime'],
#                           'dewPoint' : response['daily']['data'][0]['dewPoint'],
#                           'humidity' : response['daily']['data'][0]['humidity'],
#                           'pressure' : response['daily']['data'][0]['pressure'],
#                           'windSpeed' : response['daily']['data'][0]['windSpeed'],
#                           'windGust' : response['daily']['data'][0]['windGust'],
#                           'windGustTime' : response['daily']['data'][0]['windGustTime'],
#                           'cloudCover' : response['daily']['data'][0]['cloudCover'],
#                           'visibility' : response['daily']['data'][0]['visibility']})

#    print("Processing Record", count, "day" ' | ' , day)

#    count +=1
    
#Print("-------------------------------")
#print("Data Retrieval Complete")
#print("-------------------------------")

In [22]:
#Beginning Data Retrieval

#Processing Record 0 day |  1466186400
#Processing Record 1 day |  1466272800
#Processing Record 2 day |  1466359200
#Processing Record 3 day |  1466445600
#Processing Record 4 day |  1466532000
#Processing Record 5 day |  1466618400
#Processing Record 6 day |  1466704800
#Processing Record 7 day |  1466791200
#Processing Record 8 day |  1466877600
#Processing Record 9 day |  1466964000
#Processing Record 10 day |  1467050400
#Processing Record 11 day |  1467136800
#Processing Record 12 day |  1467223200
#Processing Record 13 day |  1467309600
#Processing Record 14 day |  1467396000
#Processing Record 15 day |  1467482400
#Processing Record 16 day |  1467568800
#Processing Record 17 day |  1467655200
#Processing Record 18 day |  1467741600
#Processing Record 19 day |  1467828000
#Processing Record 20 day |  1467914400
#Processing Record 21 day |  1468000800
#--------------------------------------
#Data Retrieval Complete
#--------------------------------------

In [23]:
# Shows DataFrame of Gathered Data
#darksky_data_1_df = pd.DataFrame(darksky_data_1)
#darksky_data_1_df.head()

In [24]:
# Exported Data to CSVs
#darksky_data_1_df.to_csv("../data/rawdata/dark_sky_1.csv", index = False, header = True)
#darksky_data_2_df.to_csv("../data/rawdata/dark_sky_1.csv", index = False, header = True)
#darksky_data_3_df.to_csv("../data/rawdata/dark_sky_3.csv", index = False, header = True)

# You May Now Run Rows
#### Merging the Dark Sky CSV Files into One DataFrame to Use For Merging Into Traffic Data

In [25]:
# Importing CSV Files to merge and join
# Importing the NYC Traffic Data
nyc_csv = os.path.join('..', 'data', 'master_clean_data', 'master_nyc_collision_data.csv')
nyc_df = pd.read_csv(nyc_csv, low_memory=False)
# Importing the Dark Sky Data
dark_sky_csv1 = os.path.join('..', 'data', 'rawdata', 'dark_sky_1.csv')
dark_sky_csv2 = os.path.join('..', 'data', 'rawdata', 'dark_sky_2.csv')
dark_sky_csv3 = os.path.join('..', 'data', 'rawdata', 'dark_sky_3.csv')
dark_sky_1 = pd.read_csv(dark_sky_csv1)
dark_sky_2 = pd.read_csv(dark_sky_csv2)
dark_sky_3 = pd.read_csv(dark_sky_csv3)

#### Converting the date for the NYC Traffic Data To Merge With the Dark Sky Weather Data

In [26]:
# Display the CSV as a DataFrame
nyc_df
nyc_df.head()

,DATE,YEAR MONTH,YEAR WEEK,YEAR,MONTH,WEEK,UNIX DATE,DEATHS,INJURIES
0,12/30/2018,2018/12,2018/52,2018,12,52,1546171200,0.0,0.0
1,12/30/2018,2018/12,2018/52,2018,12,52,1546171200,0.0,0.0
2,12/28/2018,2018/12,2018/52,2018,12,52,1545955200,0.0,0.0
3,12/17/2018,2018/12,2018/51,2018,12,51,1545065100,0.0,0.0
4,12/11/2018,2018/12,2018/50,2018,12,50,1544533200,0.0,0.0


In [27]:
# Add correctly formmated DATE column from UNIX time, and Removed the Original DATE column to perform a merge
nyc_df['Stan_Date'] = pd.to_datetime(nyc_df['UNIX DATE'],unit='s')
nyc_df['Stan_Date'] = nyc_df['Stan_Date'].dt.date
del nyc_df['DATE']

In [28]:
nyc_df = nyc_df.rename(columns={'Stan_Date': 'DATE'})
nyc_df.head(5)

,YEAR MONTH,YEAR WEEK,YEAR,MONTH,WEEK,UNIX DATE,DEATHS,INJURIES,DATE
0,2018/12,2018/52,2018,12,52,1546171200,0.0,0.0,2018-12-30
1,2018/12,2018/52,2018,12,52,1546171200,0.0,0.0,2018-12-30
2,2018/12,2018/52,2018,12,52,1545955200,0.0,0.0,2018-12-28
3,2018/12,2018/51,2018,12,51,1545065100,0.0,0.0,2018-12-17
4,2018/12,2018/50,2018,12,50,1544533200,0.0,0.0,2018-12-11


In [29]:
# Checking Columns for accuracy and reference
nyc_df.columns

Index(['YEAR MONTH', 'YEAR WEEK', 'YEAR', 'MONTH', 'WEEK', 'UNIX DATE',
       'DEATHS', 'INJURIES', 'DATE'],
      dtype='object')

In [30]:
# Resorting columns in Dataframe
nyc_sorted = ['DATE', 'YEAR MONTH', 'YEAR WEEK', 'YEAR', 'MONTH', 'WEEK', 'UNIX DATE', 'DEATHS', 'INJURIES']
nyc_df = nyc_df[nyc_sorted]
nyc_df.head()

,DATE,YEAR MONTH,YEAR WEEK,YEAR,MONTH,WEEK,UNIX DATE,DEATHS,INJURIES
0,2018-12-30,2018/12,2018/52,2018,12,52,1546171200,0.0,0.0
1,2018-12-30,2018/12,2018/52,2018,12,52,1546171200,0.0,0.0
2,2018-12-28,2018/12,2018/52,2018,12,52,1545955200,0.0,0.0
3,2018-12-17,2018/12,2018/51,2018,12,51,1545065100,0.0,0.0
4,2018-12-11,2018/12,2018/50,2018,12,50,1544533200,0.0,0.0


#### Combining Dark Sky CSV Files Generated from API into one DataFrame to Merge with NYC Traffic Data

In [31]:
# Combine the 3 CSV files into 1 file using concat
frames = [dark_sky_1, dark_sky_2, dark_sky_3]
dark_sky_df = pd.concat(frames, ignore_index=True)
dark_sky_df.reset_index
dark_sky_df.head()

,Day,Detail,Moonphase,Summary,Sunrise,Sunset,cloudCover,dewPoint,humidity,precipIntensity,...,precipProbability,pressure,temperatureMax,temperatureMaxTime,temperatureMin,temperatureMinTime,visibility,windGust,windGustTime,windSpeed
0,1388512800,partly-cloudy-day,0.97,Mostly cloudy until evening.,1388492470,1388525969,0.47,12.98,0.57,0.0000,...,0.00,1021.22,31.36,1388520000,20.95,1388491200,9.97,15.22,1388534400,2.76
1,1388599200,partly-cloudy-day,0.02,Mostly cloudy throughout the day.,1388578876,1388612417,0.44,14.62,0.56,0.0000,...,0.00,1027.54,32.61,1388606400,23.02,1388577600,10.00,10.02,1388552400,1.43
2,1388685600,fog,0.05,Foggy starting in the evening.,1388665281,1388698868,1.00,20.11,0.78,0.0003,...,0.18,1015.21,31.51,1388638800,17.65,1388721600,7.68,15.31,1388721600,5.55
3,1388772000,fog,0.09,Foggy in the morning.,1388751683,1388785319,0.54,5.02,0.71,0.0002,...,0.17,1017.60,17.06,1388775600,8.80,1388808000,5.93,19.31,1388739600,6.21
4,1388858400,partly-cloudy-night,0.12,Partly cloudy in the evening.,1388838083,1388871773,0.12,4.03,0.59,0.0000,...,0.00,1030.64,25.66,1388869200,6.26,1388829600,10.00,10.24,1388858400,1.43


In [32]:
# Check for data accuracy and overlaps
print("Dark Sky CSV has " + str(len(dark_sky_df)) + " Number of Rows")
dark_sky_df.drop_duplicates(subset=['Day'], keep= 'first', inplace = True)
print(f'Dropping Duplicate Rows')
print("Dark Sky CSV has " + str(len(dark_sky_df)) + " Number of Rows")

Dark Sky CSV has 1912 Number of Rows
Dropping Duplicate Rows
Dark Sky CSV has 1910 Number of Rows


In [33]:
# Convert the date into properly formatted date for rows
dark_sky_df['Stan_Date'] = pd.to_datetime(dark_sky_df['Day'],unit='s')
dark_sky_df['Stan_Date'] = dark_sky_df['Stan_Date'].dt.date
dark_sky_df.head(5)

,Day,Detail,Moonphase,Summary,Sunrise,Sunset,cloudCover,dewPoint,humidity,precipIntensity,...,pressure,temperatureMax,temperatureMaxTime,temperatureMin,temperatureMinTime,visibility,windGust,windGustTime,windSpeed,Stan_Date
0,1388512800,partly-cloudy-day,0.97,Mostly cloudy until evening.,1388492470,1388525969,0.47,12.98,0.57,0.0000,...,1021.22,31.36,1388520000,20.95,1388491200,9.97,15.22,1388534400,2.76,2013-12-31
1,1388599200,partly-cloudy-day,0.02,Mostly cloudy throughout the day.,1388578876,1388612417,0.44,14.62,0.56,0.0000,...,1027.54,32.61,1388606400,23.02,1388577600,10.00,10.02,1388552400,1.43,2014-01-01
2,1388685600,fog,0.05,Foggy starting in the evening.,1388665281,1388698868,1.00,20.11,0.78,0.0003,...,1015.21,31.51,1388638800,17.65,1388721600,7.68,15.31,1388721600,5.55,2014-01-02
3,1388772000,fog,0.09,Foggy in the morning.,1388751683,1388785319,0.54,5.02,0.71,0.0002,...,1017.60,17.06,1388775600,8.80,1388808000,5.93,19.31,1388739600,6.21,2014-01-03
4,1388858400,partly-cloudy-night,0.12,Partly cloudy in the evening.,1388838083,1388871773,0.12,4.03,0.59,0.0000,...,1030.64,25.66,1388869200,6.26,1388829600,10.00,10.24,1388858400,1.43,2014-01-04


In [34]:
dark_sky_df.columns

Index(['Day', 'Detail', 'Moonphase', 'Summary', 'Sunrise', 'Sunset',
       'cloudCover', 'dewPoint', 'humidity', 'precipIntensity',
       'precipIntensityMax', 'precipProbability', 'pressure', 'temperatureMax',
       'temperatureMaxTime', 'temperatureMin', 'temperatureMinTime',
       'visibility', 'windGust', 'windGustTime', 'windSpeed', 'Stan_Date'],
      dtype='object')

In [35]:
# Reorganizing columns in the Dark Sky DF to show Date at the first column
dark_sky = ['Stan_Date' ,
            'Detail',
            'humidity',
            'precipIntensity',
            'cloudCover',
            'temperatureMin',
            'visibility',
            'windSpeed',
            'Moonphase']

In [36]:
# Reforming the DataFrame to Original Name
dark_sky_df = dark_sky_df[dark_sky]
dark_sky_df.head()

,Stan_Date,Detail,humidity,precipIntensity,cloudCover,temperatureMin,visibility,windSpeed,Moonphase
0,2013-12-31,partly-cloudy-day,0.57,0.0000,0.47,20.95,9.97,2.76,0.97
1,2014-01-01,partly-cloudy-day,0.56,0.0000,0.44,23.02,10.00,1.43,0.02
2,2014-01-02,fog,0.78,0.0003,1.00,17.65,7.68,5.55,0.05
3,2014-01-03,fog,0.71,0.0002,0.54,8.80,5.93,6.21,0.09
4,2014-01-04,partly-cloudy-night,0.59,0.0000,0.12,6.26,10.00,1.43,0.12


In [37]:
# Renaming Standard Date to Date for Merging
dark_sky_df = dark_sky_df.rename(columns={'Stan_Date': 'DATE'})
dark_sky_df.head()

,DATE,Detail,humidity,precipIntensity,cloudCover,temperatureMin,visibility,windSpeed,Moonphase
0,2013-12-31,partly-cloudy-day,0.57,0.0000,0.47,20.95,9.97,2.76,0.97
1,2014-01-01,partly-cloudy-day,0.56,0.0000,0.44,23.02,10.00,1.43,0.02
2,2014-01-02,fog,0.78,0.0003,1.00,17.65,7.68,5.55,0.05
3,2014-01-03,fog,0.71,0.0002,0.54,8.80,5.93,6.21,0.09
4,2014-01-04,partly-cloudy-night,0.59,0.0000,0.12,6.26,10.00,1.43,0.12


#### Merging The Data Together

In [38]:
# Merge using left join
nyc_weather_master = pd.merge(nyc_df, dark_sky_df, on="DATE", how='left')
nyc_weather_master.head()

,DATE,YEAR MONTH,YEAR WEEK,YEAR,MONTH,WEEK,UNIX DATE,DEATHS,INJURIES,Detail,humidity,precipIntensity,cloudCover,temperatureMin,visibility,windSpeed,Moonphase
0,2018-12-30,2018/12,2018/52,2018,12,52,1546171200,0.0,0.0,partly-cloudy-day,0.69,0.0000,0.76,34.20,9.87,0.73,0.79
1,2018-12-30,2018/12,2018/52,2018,12,52,1546171200,0.0,0.0,partly-cloudy-day,0.69,0.0000,0.76,34.20,9.87,0.73,0.79
2,2018-12-28,2018/12,2018/52,2018,12,52,1545955200,0.0,0.0,rain,0.93,0.0374,1.00,42.23,5.25,3.85,0.72
3,2018-12-17,2018/12,2018/51,2018,12,51,1545065100,0.0,0.0,partly-cloudy-day,0.68,0.0000,0.88,38.11,10.00,8.54,0.32
4,2018-12-11,2018/12,2018/50,2018,12,50,1544533200,0.0,0.0,partly-cloudy-night,0.63,0.0000,0.13,25.13,10.00,1.49,0.14


In [39]:
# Verifying the amount of rows, and checking the lowest date in the column
print("The Master DataFrame Contains " + str(len(nyc_weather_master)) + " Rows")
print("The last date in the dataframe is " + str(nyc_weather_master['DATE'][1115935-1]))

The Master DataFrame Contains 1115935 Rows
The last date in the dataframe is 2014-01-01


In [40]:
# Sort Date So Earliest Time is at the Top
nyc_weather_master = nyc_weather_master.sort_values('DATE', ascending=True)
nyc_weather_master.head()

,DATE,YEAR MONTH,YEAR WEEK,YEAR,MONTH,WEEK,UNIX DATE,DEATHS,INJURIES,Detail,humidity,precipIntensity,cloudCover,temperatureMin,visibility,windSpeed,Moonphase
1115934,2014-01-01,2014/1,2014/1,2014,1,1,1388569800,0.0,0.0,partly-cloudy-day,0.56,0.0,0.44,23.02,10.0,1.43,0.02
1115661,2014-01-01,2014/1,2014/1,2014,1,1,1388593200,0.0,0.0,partly-cloudy-day,0.56,0.0,0.44,23.02,10.0,1.43,0.02
1115662,2014-01-01,2014/1,2014/1,2014,1,1,1388593500,0.0,0.0,partly-cloudy-day,0.56,0.0,0.44,23.02,10.0,1.43,0.02
1115663,2014-01-01,2014/1,2014/1,2014,1,1,1388593500,0.0,1.0,partly-cloudy-day,0.56,0.0,0.44,23.02,10.0,1.43,0.02
1115664,2014-01-01,2014/1,2014/1,2014,1,1,1388593500,0.0,1.0,partly-cloudy-day,0.56,0.0,0.44,23.02,10.0,1.43,0.02


#### Fixing an Issue where the YEAR MONTH and YEAR WEEK Columns were not correct, due to some days of the week at the beginning and end of the year being in different years. 

In [41]:
# For the Year
# Loop to fix the problem with the year and month columns where the month would be 12, and year 2015, but the year/month would be 1/2016
def f(row):
    if row['MONTH'] == 12 and row['WEEK'] == 1:
        val = row['YEAR']+1
    elif row['MONTH'] == 12 and row['WEEK'] == 53:
        val = row['YEAR']+1
    else:
        val = row['YEAR']
    return val

In [42]:
nyc_weather_master['C_YEAR'] = nyc_df.apply(f, axis=1)

In [43]:
nyc_weather_master.head()

,DATE,YEAR MONTH,YEAR WEEK,YEAR,MONTH,WEEK,UNIX DATE,DEATHS,INJURIES,Detail,humidity,precipIntensity,cloudCover,temperatureMin,visibility,windSpeed,Moonphase,C_YEAR
1115934,2014-01-01,2014/1,2014/1,2014,1,1,1388569800,0.0,0.0,partly-cloudy-day,0.56,0.0,0.44,23.02,10.0,1.43,0.02,2014
1115661,2014-01-01,2014/1,2014/1,2014,1,1,1388593200,0.0,0.0,partly-cloudy-day,0.56,0.0,0.44,23.02,10.0,1.43,0.02,2014
1115662,2014-01-01,2014/1,2014/1,2014,1,1,1388593500,0.0,0.0,partly-cloudy-day,0.56,0.0,0.44,23.02,10.0,1.43,0.02,2014
1115663,2014-01-01,2014/1,2014/1,2014,1,1,1388593500,0.0,1.0,partly-cloudy-day,0.56,0.0,0.44,23.02,10.0,1.43,0.02,2014
1115664,2014-01-01,2014/1,2014/1,2014,1,1,1388593500,0.0,1.0,partly-cloudy-day,0.56,0.0,0.44,23.02,10.0,1.43,0.02,2014


In [44]:
# For the Week
# Loop to fix the problem with the year and month columns where the month would be 12, and year 2015, but the year/month would be 1/2016
def f(row):
    if row['MONTH'] == 12 and row['WEEK'] == 53:
        val = row['WEEK']-52
    elif row['MONTH'] == 1 and row['WEEK'] == 53:
        val = row['WEEK']-52
    elif row['MONTH'] == 1 and row['WEEK'] == 52:
        val = row['WEEK']-51
    else:
        val = row['WEEK']
    return val

In [45]:
nyc_weather_master['C_WEEK'] = nyc_df.apply(f, axis=1)

In [46]:
nyc_weather_master.head()

,DATE,YEAR MONTH,YEAR WEEK,YEAR,MONTH,WEEK,UNIX DATE,DEATHS,INJURIES,Detail,humidity,precipIntensity,cloudCover,temperatureMin,visibility,windSpeed,Moonphase,C_YEAR,C_WEEK
1115934,2014-01-01,2014/1,2014/1,2014,1,1,1388569800,0.0,0.0,partly-cloudy-day,0.56,0.0,0.44,23.02,10.0,1.43,0.02,2014,1
1115661,2014-01-01,2014/1,2014/1,2014,1,1,1388593200,0.0,0.0,partly-cloudy-day,0.56,0.0,0.44,23.02,10.0,1.43,0.02,2014,1
1115662,2014-01-01,2014/1,2014/1,2014,1,1,1388593500,0.0,0.0,partly-cloudy-day,0.56,0.0,0.44,23.02,10.0,1.43,0.02,2014,1
1115663,2014-01-01,2014/1,2014/1,2014,1,1,1388593500,0.0,1.0,partly-cloudy-day,0.56,0.0,0.44,23.02,10.0,1.43,0.02,2014,1
1115664,2014-01-01,2014/1,2014/1,2014,1,1,1388593500,0.0,1.0,partly-cloudy-day,0.56,0.0,0.44,23.02,10.0,1.43,0.02,2014,1


In [47]:
nyc_weather_master.columns

Index(['DATE', 'YEAR MONTH', 'YEAR WEEK', 'YEAR', 'MONTH', 'WEEK', 'UNIX DATE',
       'DEATHS', 'INJURIES', 'Detail', 'humidity', 'precipIntensity',
       'cloudCover', 'temperatureMin', 'visibility', 'windSpeed', 'Moonphase',
       'C_YEAR', 'C_WEEK'],
      dtype='object')

In [48]:
nyc_weather_master.dtypes

DATE                object
YEAR MONTH          object
YEAR WEEK           object
YEAR                 int64
MONTH                int64
WEEK                 int64
UNIX DATE            int64
DEATHS             float64
INJURIES           float64
Detail              object
humidity           float64
precipIntensity    float64
cloudCover         float64
temperatureMin     float64
visibility         float64
windSpeed          float64
Moonphase          float64
C_YEAR               int64
C_WEEK               int64
dtype: object

In [49]:
nyc_weather_master['DATE'] = nyc_weather_master['DATE'].astype(np.datetime64)

In [50]:
nyc_weather_master.dtypes

DATE               datetime64[ns]
YEAR MONTH                 object
YEAR WEEK                  object
YEAR                        int64
MONTH                       int64
WEEK                        int64
UNIX DATE                   int64
DEATHS                    float64
INJURIES                  float64
Detail                     object
humidity                  float64
precipIntensity           float64
cloudCover                float64
temperatureMin            float64
visibility                float64
windSpeed                 float64
Moonphase                 float64
C_YEAR                      int64
C_WEEK                      int64
dtype: object

In [51]:
nyc_weather_master.head()

,DATE,YEAR MONTH,YEAR WEEK,YEAR,MONTH,WEEK,UNIX DATE,DEATHS,INJURIES,Detail,humidity,precipIntensity,cloudCover,temperatureMin,visibility,windSpeed,Moonphase,C_YEAR,C_WEEK
1115934,2014-01-01,2014/1,2014/1,2014,1,1,1388569800,0.0,0.0,partly-cloudy-day,0.56,0.0,0.44,23.02,10.0,1.43,0.02,2014,1
1115661,2014-01-01,2014/1,2014/1,2014,1,1,1388593200,0.0,0.0,partly-cloudy-day,0.56,0.0,0.44,23.02,10.0,1.43,0.02,2014,1
1115662,2014-01-01,2014/1,2014/1,2014,1,1,1388593500,0.0,0.0,partly-cloudy-day,0.56,0.0,0.44,23.02,10.0,1.43,0.02,2014,1
1115663,2014-01-01,2014/1,2014/1,2014,1,1,1388593500,0.0,1.0,partly-cloudy-day,0.56,0.0,0.44,23.02,10.0,1.43,0.02,2014,1
1115664,2014-01-01,2014/1,2014/1,2014,1,1,1388593500,0.0,1.0,partly-cloudy-day,0.56,0.0,0.44,23.02,10.0,1.43,0.02,2014,1


In [52]:
#Remove Columns that are incorrect and uneeded, and sort 
final_df = nyc_weather_master[['DATE',
                    'C_YEAR',
                    'C_WEEK',
                    'DEATHS',
                    'INJURIES',
                    'Detail',
                    'humidity',
                    'precipIntensity',
                    'cloudCover',
                    'temperatureMin',
                    'visibility',
                    'windSpeed',
                    'Moonphase']]

In [53]:
final_df.columns

Index(['DATE', 'C_YEAR', 'C_WEEK', 'DEATHS', 'INJURIES', 'Detail', 'humidity',
       'precipIntensity', 'cloudCover', 'temperatureMin', 'visibility',
       'windSpeed', 'Moonphase'],
      dtype='object')

In [54]:
# Export File to CSV to begin Analysis
final_df.to_csv(os.path.join('..', 'data', 'master_clean_data', 'nyc_final_merged.csv'), index=False, header=True)